In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from flask import Flask, request, jsonify

## Import Pickle

In [3]:
import pickle

In [4]:
file = open('rain_model.pkl', 'rb')
rain_model = pickle.load(file)
file.close()

In [5]:
file = open('wind_model.pkl', 'rb')
wind_model = pickle.load(file)
file.close()

In [6]:
file = open('temp_model.pkl', 'rb')
temp_model = pickle.load(file)
file.close()

In [7]:
file = open('avg_air_pressure.pkl', 'rb')
air_pressure_avg = pickle.load(file)
file.close()

file = open('avg_humidity.pkl', 'rb')
humidity_avg = pickle.load(file)
file.close()

file = open('monthly_avg_wind_speed.pkl', 'rb')
wind_speed_avg = pickle.load(file)
file.close()

In [8]:
print(wind_speed_avg)

Month
1     4.731984
2     4.613775
3     4.225179
4     4.088561
5     3.308384
6     3.098317
7     2.891623
8     2.954435
9     3.263625
10    3.944265
11    4.333734
12    4.671007
Name: Surface Air Temp (K), dtype: float64


In [9]:
#Define a function that takes two dates and produces an array of dates between them
def split_dates(start, end):
    dates = pd.date_range(start, end)
    return dates

## Get/Post

In [10]:
app = Flask(__name__)

In [11]:
def get_prediction(date, latitude, longitude):
    
        month = date.month
        day = date.day
        year = date.year
        hour = date.hour
        
        rain_df = {
        'Humidity':[humidity_avg[month]],
        'Pressure (Pa)': [air_pressure_avg[month]],
        'Year':[year],
        'Month':[month],
        'Day':[day],
        'Hour':[hour],
        'Longitude':[longitude],
        'Latitude':[latitude]}
    
        wind_df = {        
        'Surface Air Temp (K)':[0],
        'Humidity (g/kg)': [0],
        'Year':[year],
        'Month':[month],
        'Day':[day],
        'Hour':[hour],
        'Longitude':[longitude],
        'Latitude':[latitude]}
   
        temp_df = {
        'Surface Wind Speed (m/s)':[0],
        'Humidity (g/kg)': [0],
        'Year':[year],
        'Month':[month],
        'Day':[day],
        'Hour':[hour],
        'Longitude':[longitude],
        'Latitude':[latitude] }

        rain_df1 = pd.DataFrame(rain_df)
        wind_df = pd.DataFrame(wind_df)
        temp_df = pd.DataFrame(temp_df)

        rain = rain_model.predict(rain_df1)[0]
        wind = wind_model.predict(wind_df)[0]
        temp = temp_model.predict(temp_df)[0]

        prediction = {'Date': date, 'Rain': rain, 'Wind':wind, 'Temp':temp}
        return prediction

## Method 1 Range of Date

In [12]:
@app.route("/request", methods=["POST"])
def predict2():

    data = request.get_json()

    latitude = data['Latitude']
    longitude = data['Longitude']
    
    start_date = data['Start_Date']
    end_date = data['End_Date']
    date_list = split_dates(start_date, end_date)

    pred_list = []
    for each in date_list:
        prediction = get_prediction(each, latitude, longitude)
        pred_list.append(prediction)
    
    print(pred_list)
    return jsonify(pred_list)

## Method 2, A Single Date

In [ ]:
@app.route("/request_date", methods=["POST"])
def predict_single():

    data = request.get_json(force=True)

    latitude = data['Latitude']
    longitude = data['Longitude']
    
    date = data['Date']

    prediction = get_prediction(date, latitude, longitude)

    
    print(prediction)
    return jsonify(prediction)

## Method 3: A single Date, seperate

In [14]:
@app.route("/request_date_sep", methods=["POST"])
def predict_single2():

    data = request.get_json(force=True)

    latitude = data['Latitude']
    longitude = data['Longitude']
    
    day = data['Day']
    month = data['Month']
    year = data['Year']
    hour = data['Hour']

    rain_df = {
        'Humidity':[humidity_avg[month]],
        'Pressure (Pa)': [air_pressure_avg[month]],
        'Year':[year],
        'Month':[month],
        'Day':[day],
        'Hour':[hour],
        'Longitude':[longitude],
        'Latitude':[latitude]}
    
    wind_df = {        
        'Surface Air Temp (K)':[0],
        'Humidity (g/kg)': [0],
        'Year':[year],
        'Month':[month],
        'Day':[day],
        'Hour':[hour],
        'Longitude':[longitude],
        'Latitude':[latitude]}
   
    temp_df = {
        'Surface Wind Speed (m/s)':[0],
        'Humidity (g/kg)': [0],
        'Year':[year],
        'Month':[month],
        'Day':[day],
        'Hour':[hour],
        'Longitude':[longitude],
        'Latitude':[latitude] }

    rain_df1 = pd.DataFrame(rain_df)
    wind_df = pd.DataFrame(wind_df)
    temp_df = pd.DataFrame(temp_df)

    rain = rain_model.predict(rain_df1)[0]
    wind = wind_model.predict(wind_df)[0]
    temp = temp_model.predict(temp_df)[0]

    prediction = { 'Rain': rain, 'Wind':wind, 'Temp':temp}
    print(prediction)

    return jsonify(prediction)



In [17]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Oct/2025 10:58:23] "POST /request HTTP/1.1" 200 -


[{'Date': Timestamp('2025-10-01 08:00:00'), 'Rain': 3.3509059887684364e-05, 'Wind': 102754.68824969916, 'Temp': 275.77054762668763}, {'Date': Timestamp('2025-10-02 08:00:00'), 'Rain': 3.3614009596614255e-05, 'Wind': 102754.06813841716, 'Temp': 275.76287720790276}, {'Date': Timestamp('2025-10-03 08:00:00'), 'Rain': 3.3718959305544145e-05, 'Wind': 102753.44802713516, 'Temp': 275.7552067891179}, {'Date': Timestamp('2025-10-04 08:00:00'), 'Rain': 3.3823909014474036e-05, 'Wind': 102752.82791585318, 'Temp': 275.747536370333}, {'Date': Timestamp('2025-10-05 08:00:00'), 'Rain': 3.3928858723403926e-05, 'Wind': 102752.20780457118, 'Temp': 275.73986595154815}, {'Date': Timestamp('2025-10-06 08:00:00'), 'Rain': 3.403380843233371e-05, 'Wind': 102751.58769328918, 'Temp': 275.7321955327633}]


In [ ]:
# ## Example 
# date1 = '2023-01-01'
# date2 = '2023-01-03'
# date_list = split_dates(date1, date2)

# pred_list = []
# for each in date_list:
#         prediction = get_prediction(each, 50, -80)
#         pred_list.append(prediction)

# print(pred_list)

[{'Date': Timestamp('2023-01-01 00:00:00'), 'Rain': 1.861835249021603e-05, 'Wind': 99796.17080719717, 'Temp': 263.85566124802926}, {'Date': Timestamp('2023-01-02 00:00:00'), 'Rain': 1.872330219914592e-05, 'Wind': 99795.55069591517, 'Temp': 263.8479908292444}, {'Date': Timestamp('2023-01-03 00:00:00'), 'Rain': 1.882825190807581e-05, 'Wind': 99794.93058463317, 'Temp': 263.8403204104596}]


In [ ]:
# ## Post will have 
# # Longitude/Latitude
# # Hour/Day/Month/Year
# @app.route("/request", methods=["POST"])
# def predict1():

#     data = request.get_json()

#     latitude = data['Latitude']
#     longitude = data['Longitude']
    
#     start_date = data['Start_Date']
#     end_date = data['End_Date']
#     date_list = split_dates(start_date, end_date)

#     prediction_list = []
#     #for each Date, create a dataframe, and predict the rain wind and temp
#     for each in date_list:
#         month = data['Month']
#         day = data['Day']
#         year = data['Year']
#         hour = data['Hour']


#         rain_df = {
#         'Humidity':[humidity_avg[month -1]],
#         'Pressure (Pa)': [air_pressure_avg[month -1]],
#         'Year':[year],
#         'Month':[month],
#         'Day':[day],
#         'Hour':[hour],
#         'Longitude':[longitude],
#         'Latitude':[latitude]}
    
#         wind_df = {        
#         'Surface Air Temp (K)':[0],
#         'Humidity (g/kg)': [0],
#         'Year':[year],
#         'Month':[month],
#         'Day':[day],
#         'Hour':[hour],
#         'Longitude':[longitude],
#         'Latitude':[latitude]}
   
#         temp_df = {
#         'Surface Wind Speed (m/s)':[0],
#         'Humidity (g/kg)': [0],
#         'Year':[year],
#         'Month':[month],
#         'Day':[day],
#         'Hour':[hour],
#         'Longitude':[longitude],
#         'Latitude':[latitude] }

#         rain_df1 = pd.DataFrame(rain_df)
#         wind_df = pd.DataFrame(wind_df)
#         temp_df = pd.DataFrame(temp_df)

#         rain = rain_model.predict(rain_df1)[0]
#         wind = wind_model.predict(wind_df)[0]
#         temp = temp_model.predict(temp_df)[0]

#         prediction = {'Date': each, 'Rain': rain, 'Wind':wind, 'Temp':temp}
#         prediction_list.append(prediction)
#         #end loop

#     return jsonify({ 'Predictions' : prediction_list})




In [ ]:
# date1 = '2023-01-01T08:00:00'
# date2 = '2023-01-02T08:00:00'
# dates = split_dates(date1, date2)
# # print(dates)


DatetimeIndex(['2023-01-01 08:00:00', '2023-01-02 08:00:00'], dtype='datetime64[ns]', freq='D')


In [ ]:
# pred_list = []

# for each in dates:
#         month = each.month
#         day = each.day
#         year = each.year
#         hour = each.hour


#         rain_df = {
#         'Humidity':[0],
#         'Pressure (Pa)': [0],
#         'Year':[year],
#         'Month':[month],
#         'Day':[day],
#         'Hour':[hour],
#         'Longitude':[50],
#         'Latitude':[-80]}
    
#         wind_df = {        
#         'Surface Air Temp (K)':[0],
#         'Humidity (g/kg)': [0],
#         'Year':[year],
#         'Month':[month],
#         'Day':[day],
#         'Hour':[hour],
#         'Longitude':[50],
#         'Latitude':[-80]}
   
#         temp_df = {
#         'Surface Wind Speed (m/s)':[0],
#         'Humidity (g/kg)': [0],
#         'Year':[year],
#         'Month':[month],
#         'Day':[day],
#         'Hour':[hour],
#         'Longitude':[50],
#         'Latitude':[-80] }

#         rain_df1 = pd.DataFrame(rain_df)
#         wind_df = pd.DataFrame(wind_df)
#         temp_df = pd.DataFrame(temp_df)

#         # rain = rain_model.predict(rain_df1)[0]
#         wind = wind_model.predict(wind_df)[0]
#         temp = temp_model.predict(temp_df)[0]

#         prediction = {'Date': each, 'Rain': rain, 'Wind':wind, 'Temp':temp}
#         pred_list.append(prediction)
    
# print(pred_list)

[{'Date': Timestamp('2023-01-01 08:00:00'), 'Rain': 0.001246843873389896, 'Wind': 181143.9715257681, 'Temp': 492.55329476651434}, {'Date': Timestamp('2023-01-02 08:00:00'), 'Rain': 0.001246948823098826, 'Wind': 181143.3514144861, 'Temp': 492.5456243477295}]


In [ ]:
# for each in dates:
#     print(each.day)
#     print(each.month)
#     # print(each.year)
#     print(each.hour)

1
1
2023
0
2
1
2023
0
3
1
2023
0
4
1
2023
0
5
1
2023
0
6
1
2023
0
7
1
2023
0
8
1
2023
0
9
1
2023
0
10
1
2023
0
